# Measuring the distance to a Supernova

Using the provided notebook, determine how to find the distance to a supernova by fitting a model to its lightcurve. Create a figure to explain the model you fit, the parameters you chose, and the distance you determined. Comment your code throughout, such that another team can quickly understand each step in your code.

This notebook uses iPython cells to do things like read in data, plot, and calculate. We have provided some code cells to get you started. You can add to existing cells or insert your own with the (+ CODE) box. Comments can be added using a # symbol or by using separate (+ TEXT) boxes.

This first cell imports some useful packages you may want to use. To run this cell, click inside it and hit shift+enter

In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
!pip install astropy
import astropy
from astropy import units as u
from astropy import cosmology
from astropy.cosmology import WMAP9 as cosmo
from astropy.cosmology import z_at_value
!pip install sncosmo
import sncosmo
!pip install iminuit
import iminuit
from scipy import optimize

In this notebook, you'll measure the observed brightness, or **flux**, of a type Ia supernova, and use that to calculate the distance to that supernova. Type Ia supernovae are unique objects in this regard, because we have independent knowledge of their intrinsic  **luminosity**. 

# Exploring the Data

Next, upload the supernova light curve data file
This is table 10 from http://cdsarc.u-strasbg.fr/viz-bin/qcat?J/A+A/523/A7

In [0]:
# Remove the directory if it already exists
!rm -rf carnegiepdp2018
# Download data and models
!git clone https://github.com/deckerkf/carnegiepdp2018.git
# Add this so we can import code
import sys
sys.path.insert(0,"carnegiepdp2018")

This next cell reads in the supernova light curve data file with pandas

In [0]:
data = pd.read_csv('carnegiepdp2018/sntable.dat') #read in the data file

Next, take a look at the data. Create a new code cell below, type "data", and hit shift-enter to see a table of the data.

This datafile contains flux measurements for a set of supernova at different times and in different color filters. The times are in modified julian date (MJD) days, which are the fractional number of days since midnight Nov 17, 1858. The flux measurements are in "analog data units" (ADU) scaled from a zero-point of 30 mag, indicated by the "zp" column. In practice, this means the fluxes are on a linear scale. The supernova model we will use later will take these units and this format into account.

The pandas code to select a single supernova and a single filter looks like the following:

In [0]:
SN_1 = data[data["name"] == "03D1ar"]
SN_1g = SN_1[SN_1["filter"] == "sdssg"]

Look at the data now in SN_1g to verify this has done what you expect

We can also look at the data graphically, using code like this:

In [0]:
plt.plot(SN_1g['mjd'],SN_1g['flux'],marker='*',linestyle='none',markersize=10)
plt.xlabel('Time (days, MJD)')
plt.ylabel('Flux')
plt.show()

Some useful resources for plotting with matplotlib are [the matplotlib gallery](https://matplotlib.org/gallery.html) and [pyplot reference](https://matplotlib.org/2.1.1/api/_as_gen/matplotlib.pyplot.plot.html).

Individual elements of pandas datatables can be read using ".iloc":

In [0]:
print(SN_1g.iloc[0]['mjd']) #print the first element of the column named "mjd"

# Standard Candles

Next, look at the lightcurves for the supernovae 04D1hd and 06D4dh. The distances to these supernovae are 2002 Mpc and 1591 Mpc, respectively. Can these 2 lightcurves be used to measure distance? Why or why not? Explain using a plot. Comment your code throughout, such that another team can quickly understand each step in your code.

Swap code with your partner and talk through your solutions.

# Exploring the supernova model

Next, we'll use a supernova model called MLCS2K2 and a tool called SNcosmo to fit to the data

This next block is a function which will take your input data, as well as an array of times, and a list of model parameters, and return model fluxes for each of those times as well as the implied distance given those parameters. If you would like to come back to this section to edit the model function, more information about the model and sncosmo can be found [here](https://sncosmo.readthedocs.io).

In [0]:
def sn_model(dat,time,t0,f0,delta):
    """
    Generate a supernova light curve using sncosmo and the MLCS2K2 models
    inputs:
      dat: pandas table with supernova data for your selected object and filter
      time: time array (mjd) to calculate model spectra
      t0: time (mjd) of peak flux
      f0: close to your estimate of the peak flux (in the same units as provided in the data table)
          the model peak flux will vary somewhat from this depending on the other parameters and the filter
      delta: shape parameter; 0 is a good guess, but try a range from -0.5 to 1.8 to see how this behaves
    outputs:
      flux_model: the model fluxes at each time in the input array, also in zero-point=25mag flux units
      distance: the implied distance to the supernova given the input parameters
    """
    table = astropy.table.Table.from_pandas(dat) # convert from a pandas table to an astropy table
    filt = dat.iloc[0]['filter']
    zpsys = SN_1g.iloc[0]['zpsys']
    zp = SN_1g.iloc[0]['zp']
    model = sncosmo.Model('mlcs2k2')
    Mabs = -19.504+0.736*delta+0.182*delta**2 + 5*np.log10(69/65)
    mag0 = zp-2.5*np.log10(f0)
    dmod = mag0-Mabs
    zz = z_at_value(cosmo.distmod, dmod*u.mag)
    model.parameters[0] = zz
    model.parameters[1] = t0
    model.parameters[3] = delta
    model.set_source_peakabsmag(Mabs, filt,zpsys)
    model.set_source_peakmag(mag0, filt, zpsys)
    flux_model = model.bandflux(filt, time ,zpsys=zpsys,zp=zp)
    distance = cosmo.luminosity_distance(zz)
    return flux_model, distance

Start by exploring the model parameters to see how varying them changes the shape of the lightcurve

# Fitting the supernova model by hand

Next, pick a supernova dataset, and experiment with finding the best-fit parameters by hand. Some suggestions for SN dataset choices are: 03D1ar, 04D1hd,04D3fk, or 05D4bm.

Swap code with your partner and talk through your solutions.

# Automate fitting your model

(optional) Use `scipy.optimize.curve_fit` to determine the best-fit parameters automatically.
Create a figure to explain the model you fit, the parameters you chose, and the distance you determined. Comment your code throughout, such that another team can quickly understand each step in your code.

In [0]:
#use this to see what inputs curve_fit needs and what outputs it generates
?optimize_curve_fit

Curve_fit requires the function to be fit to be defined in a specific way, so use the following wrapper function

In [0]:
# define a wrapper function to use with curve_fit
def fitfn(xdata, *params):
  return sn_model(sndat, xdata, *params)[0]

Swap code with your partner and talk through your solutions.